# Receiving C++ Statistical Analysis and Transforming to ArcGIS Online #

 First, import ArcGIS 

In [4]:

from arcgis.widgets import MapView
from arcgis.features import FeatureLayer
from arcgis.gis import GIS
gis = GIS()

ModuleNotFoundError: No module named 'arcgis'

In [2]:
m = gis.map('Ridgefield, CT')
m

NameError: name 'gis' is not defined

In [5]:
m.basemaps

['dark-gray-vector',
 'gray-vector',
 'hybrid',
 'oceans',
 'osm',
 'satellite',
 'streets-navigation-vector',
 'streets-night-vector',
 'streets-relief-vector',
 'streets-vector',
 'terrain',
 'topo-vector']

In [6]:
for bm in m.basemaps[::10]:
    print(m.basemap)
    m.basemap = bm    

default
dark-gray-vector


In [7]:
m

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)